Validation: 1. Model works for multi DC, multi customer locations
2. Testing yet to be done for multi port, multi DC and multi customer locations.

Install libraries

In [ ]:
!pip install geopy
!pip install openrouteservice
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 67.5 MB/s eta 0:00:00


Import libraries

In [425]:
from geopy.geocoders import Nominatim
import pandas as pd
import time
import openrouteservice
import math


Load port, DC and customer sheets.

In [426]:
df_port_city=pd.read_excel('Port.xlsx')
df_DC_city=pd.read_excel('DC.xlsx')
df_customer_city=pd.read_excel('Customer.xlsx')

In [427]:
print(df_port_city)
print(df_DC_city)
print(df_customer_city)

          City
0  NHAVA SHEVA
         City  Fixed Cost
0   New Delhi       10000
1  Coimbatore       20000
2     Kolkata     2000000
3      Mumbai     2000000
          City  Demand
0    Bengaluru    3000
1      Madurai    3200
2        Kochi    3300
3      Chennai    3400
4       Mysuru    3500
5       Meerut    3600
6      Aligarh    3700
7         Agra    3800
8   Chandigarh    3900
9      Bhiwani    4000
10      Jaipur    4100
11      Kanpur    4200


Defining dataframes for port, DC and customer to hold coordinates data.

In [428]:
df_coordinate_port_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_DC_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_customer_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Demand'])

Finding coordinates for ports

In [429]:
geolocator = Nominatim(user_agent="your_email_or_app_name")
for i in range(0, len(df_port_city)):
  location = geolocator.geocode(df_port_city['City'].iloc[i])
  if location:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], location.latitude, location.longitude]
  else:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], None, None]

Exporting port coordinates to excel

In [430]:
df_coordinate_port_city.to_excel('coordinates_port_city.xlsx', index=False)

Finding coordinates for DC

In [431]:
import pandas as pd
from geopy.geocoders import Nominatim

# Assuming df_coordinate_DC_city was initialized as:
# df_coordinate_DC_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

# Modify the code as follows:
# Define columns for df_coordinate_DC_city if it doesn't exist
if df_coordinate_DC_city.empty:
  df_coordinate_DC_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Fixed Cost'])

for i in range(0, len(df_DC_city)):
  location = geolocator.geocode(df_DC_city['City'].iloc[i])
  if location:
    # Assign values to the row, including the 'Fixed Cost' column
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], location.latitude, location.longitude, df_DC_city['Fixed Cost'].iloc[i]]
  else:
    # Assign values to the row, including the 'Fixed Cost' column and None for coordinates
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], None, None, df_DC_city['Fixed Cost'].iloc[i]]

Exporting DC coordinates to excel

In [432]:
df_coordinate_DC_city.to_excel('coordinates_DC_city.xlsx', index=False)

Finding coordinates for customer

In [433]:
geolocator = Nominatim(user_agent="your_email_or_app_name")
# Increased timeout to 10 seconds
for i in range(0, len(df_customer_city)):
  try:
    location = geolocator.geocode(df_customer_city['City'].iloc[i], timeout=10)
    if location:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], location.latitude, location.longitude,df_customer_city['Demand'].iloc[i]]
    else:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
    #time.sleep(2) # Wait 2 seconds between requests to avoid rate limiting
  except GeocoderTimedOut:
    print("Geocoding timed out for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except GeocoderUnavailable:
    print("Geocoder service unavailable for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except Exception as e:
    print("Error geocoding city:", df_customer_city['City'].iloc[i], "-", e)
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]

Exporting coordintates of customers to excel.

In [434]:
df_coordinate_customer_city.to_excel('coordinate_customer_city.xlsx', index=False)

Define a new dataframe

In [435]:
print(df_coordinate_customer_city)

          City   Latitude  Longitude  Demand
0    Bengaluru  12.976794  77.590082    3000
1      Madurai   9.926115  78.114098    3200
2        Kochi   9.967428  76.245444    3300
3      Chennai  13.083694  80.270186    3400
4       Mysuru  12.305183  76.655361    3500
5       Meerut  28.996330  77.706192    3600
6      Aligarh  27.876107  78.135815    3700
7         Agra  27.175255  78.009816    3800
8   Chandigarh  30.729844  76.784146    3900
9      Bhiwani  28.793170  76.139128    4000
10      Jaipur  26.915458  75.818982    4100
11      Kanpur  26.460914  80.321759    4200


In [436]:
print(df_coordinate_port_city)

          City   Latitude  Longitude
0  NHAVA SHEVA  18.945031  72.949423


In [437]:
print(df_coordinate_DC_city)

         City   Latitude  Longitude  Fixed Cost
0   New Delhi  28.613895  77.209006       10000
1  Coimbatore  11.001812  76.962842       20000
2     Kolkata  22.572646  88.363895     2000000
3      Mumbai  19.054999  72.869203     2000000


Defining two new dataframes for distance calculations

In [438]:
df_distance_port_DC=pd.DataFrame(columns=['Source','Destination','Distance'])
df_distance_DC_customer=pd.DataFrame(columns=['Source','Destination','Distance'])

Defining a function which will calculate the distance

In [439]:
def distance(lat1, lon1, lat2, lon2):
  client = openrouteservice.Client(key='5b3ce3597851110001cf62489d683e3d831942f3adab13e6a2e5a7dc')

# Coordinates of New Delhi and Kolkata (longitude, latitude)
  source = (lon1, lat1)
  destination = (lon2, lat2)
  routes = client.directions(coordinates=[source, destination],profile='driving-hgv', format='geojson')

  distance_meters = routes['features'][0]['properties']['segments'][0]['distance']
  distance_km = distance_meters / 1000
  #print(f"Road distance between New Delhi and Kolkata is {distance_km:.2f} km")
  return distance_km

Estimating distance for port to DC

In [440]:
k=0
for j in range(0,len(df_coordinate_port_city)):
  for i in range(0,len(df_coordinate_DC_city)):
    dist=distance(df_coordinate_port_city['Latitude'].iloc[j], df_coordinate_port_city['Longitude'].iloc[j], df_coordinate_DC_city['Latitude'].iloc[i], df_coordinate_DC_city['Longitude'].iloc[i])
    df_distance_port_DC.loc[k]=[df_coordinate_port_city['City'].iloc[j],df_coordinate_DC_city['City'].iloc[i],dist]
    k=k+1


In [441]:
print(df_distance_port_DC)
print(df_distance_port_DC['Distance'][0])

        Source Destination   Distance
0  NHAVA SHEVA   New Delhi  1482.7279
1  NHAVA SHEVA  Coimbatore  1324.2070
2  NHAVA SHEVA     Kolkata  1932.8627
3  NHAVA SHEVA      Mumbai    47.8468
1482.7278999999999


Estimating distance for DC to customer

In [442]:
k=0
for j in range(0,len(df_coordinate_DC_city)):
  for i in range(0,len(df_coordinate_customer_city)):
    dist=distance(df_coordinate_DC_city['Latitude'].iloc[j], df_coordinate_DC_city['Longitude'].iloc[j], df_coordinate_customer_city['Latitude'].iloc[i], df_coordinate_customer_city['Longitude'].iloc[i])
    df_distance_DC_customer.loc[k]=[df_coordinate_DC_city['City'].iloc[j],df_coordinate_customer_city['City'].iloc[i],dist]
    k=k+1

/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rat

In [443]:
print(df_distance_DC_customer)
df_distance_port_DC.to_excel('distance_port_DC.xlsx', index=False)
df_distance_DC_customer.to_excel('distance_DC_customer.xlsx', index=False)

        Source Destination   Distance
0    New Delhi   Bengaluru  2110.0153
1    New Delhi     Madurai  2542.9761
2    New Delhi       Kochi  2650.2621
3    New Delhi     Chennai  2173.4354
4    New Delhi      Mysuru  2245.2512
5    New Delhi      Meerut    83.1616
6    New Delhi     Aligarh   157.6321
7    New Delhi        Agra   200.9726
8    New Delhi  Chandigarh   258.5782
9    New Delhi     Bhiwani   125.2304
10   New Delhi      Jaipur   272.6346
11   New Delhi      Kanpur   450.0073
12  Coimbatore   Bengaluru   338.9688
13  Coimbatore     Madurai   228.0159
14  Coimbatore       Kochi   197.8971
15  Coimbatore     Chennai   513.7791
16  Coimbatore      Mysuru   208.4256
17  Coimbatore      Meerut  2500.2858
18  Coimbatore     Aligarh  2355.9328
19  Coimbatore        Agra  2264.1069
20  Coimbatore  Chandigarh  2772.9097
21  Coimbatore     Bhiwani  2508.8106
22  Coimbatore      Jaipur  2259.6561
23  Coimbatore      Kanpur  2143.8341
24     Kolkata   Bengaluru  1887.1274
25     Kolka

In [496]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("MyModel")



In [497]:
# Create an environment with your WLS license
params = {
"WLSACCESSID": '803f8182-ebee-4591-8370-b64a82f775ea',
"WLSSECRET": '7696ca12-c44b-46e1-9875-a948ff7c3fdf',
"LICENSEID": 2574149,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2574149
Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com


Defining known variables

In [498]:
c = 0.1
r = 0
for i in range(0, len(df_customer_city)):
  r = r + df_customer_city['Demand'][i]
print(r)

43700


Defining unknown variables

In [499]:
x_var = {}
y_var = {}
z_var = {}

In [500]:
z = 0
for i in range(0, len(df_DC_city)):
  z_var[f"z[{i}]"] = model.addVar(vtype=GRB.BINARY, name=f"z[{i}]")
  z = z + z_var[f"z[{i}]"]
print(len(z_var))
print(z_var)

4
{'z[0]': <gurobi.Var *Awaiting Model Update*>, 'z[1]': <gurobi.Var *Awaiting Model Update*>, 'z[2]': <gurobi.Var *Awaiting Model Update*>, 'z[3]': <gurobi.Var *Awaiting Model Update*>}


In [501]:
x = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_port_city)):
    x_var[f"x[{i+j}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"x[{i+j}]")
    x = x + x_var[f"x[{i+j}]"]*z_var[f"z[{j}]"]
print(len(x_var))
print(x_var)
print(x)

4
{'x[0]': <gurobi.Var *Awaiting Model Update*>, 'x[1]': <gurobi.Var *Awaiting Model Update*>, 'x[2]': <gurobi.Var *Awaiting Model Update*>, 'x[3]': <gurobi.Var *Awaiting Model Update*>}
0.0 + [ <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> ]


In [502]:
y = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_DC_city)*len(df_customer_city)):
    y_var[f"y[{i}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"y[{i}]")
    y = y + y_var[f"y[{i}]"]*z_var[f"z[{j}]"]
print(len(y_var))
print(y)

48
0.0 + [ <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *A

Constraints

Demand of each customer is exactly met from DCs.

In [503]:
for j in range(0, len(df_customer_city)):
  a = 0
  for i in range(0, len(df_DC_city)):
    a = a + y_var[f"y[{j+len(df_customer_city)*i}]"]
  model.addConstr(a == df_customer_city['Demand'][j])

1. Outgoing from ports = total demand
2. Outgoing from DC = total demand
3. atleast one DC is opened

In [504]:
model.addConstr(x == r, "Demand_x")
model.addConstr(y == r, "Demand_y")
model.addConstr(z >= 1, "Z_non_zero")

<gurobi.Constr *Awaiting Model Update*>

Total incoming at a DC = Total outgoing from a DC

In [505]:
incoming = 0
outgoing = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    incoming = incoming + x_var[f"x[{i+len(df_DC_city)*j}]"]
  for k in range(0, len(df_customer_city)):
    outgoing = outgoing + y_var[f"y[{k+len(df_customer_city)*i}]"]
  model.addConstr(incoming == outgoing)

In [506]:
print(x)

0.0 + [ <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> ]


Pre-work for objective function

In [507]:
p = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    p = p + df_distance_port_DC['Distance'][i+len(df_DC_city)*j]*x_var[f"x[{i+len(df_DC_city)*j}]"]*z_var[f"z[{i}]"]
p = p*c #This multiplication is fine as X is the linear expression before z is applied
print(p)

0.0 + [ 148.27279 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 132.4207 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 193.28627 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 4.784680000000001 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> ]


In [508]:
q = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_customer_city)):
    print(df_distance_DC_customer['Distance'][len(df_customer_city)*i+j])
    q = q + df_distance_DC_customer['Distance'][len(df_customer_city)*i+j]*y_var[f"y[{len(df_customer_city)*i+j}]"]*z_var[f"z[{i}]"]
q = q*c
print(q)

2110.0153
2542.9761000000003
2650.2621
2173.4354
2245.2512
83.1616
157.6321
200.9726
258.57820000000004
125.23039999999999
272.6346
450.0073
338.9688
228.0159
197.8971
513.7791
208.4256
2500.2857999999997
2355.9327999999996
2264.1068999999998
2772.9097
2508.8106000000002
2259.6561
2143.8341
1887.1273999999999
2109.763
2297.974
1663.9556
2090.689
1461.6813
1324.7031000000002
1302.6391999999998
1727.5482
1599.7848999999999
1535.13
1028.698
994.2570999999999
1420.8811
1355.3947
1251.9735
1050.9386000000002
1542.7494
1491.6061000000002
1401.5558
1679.9323
1415.8332
1166.6787
1335.2829
0.0 + [ 211.00153 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 254.29761000000005 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 265.02621 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 217.34354 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 224.52512000000002 <gurobi.Var *Awaiting Mo

In [509]:
s = 0
for i in range(0, len(df_DC_city)):
  s = s + df_DC_city['Fixed Cost'][i]*z_var[f"z[{i}]"]
print(s)

10000.0 <gurobi.Var *Awaiting Model Update*> + 20000.0 <gurobi.Var *Awaiting Model Update*> + 2000000.0 <gurobi.Var *Awaiting Model Update*> + 2000000.0 <gurobi.Var *Awaiting Model Update*>


Objective function

In [510]:
objective =  p + q + s

In [511]:
model.setObjective(objective, GRB.MINIMIZE)

In [512]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com
Optimize a model with 17 rows, 200 columns and 182 nonzeros
Model fingerprint: 0x24540545
Model has 52 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 0 continuous, 200 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+04, 2e+06]
  QObjective range [1e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
  QRHS range       [4e+04, 4e+04]
Presolve time: 0.00s
Presolved: 483 rows, 896 columns, 1451 nonzeros
Presolved model has 464 SOS constraint(s)
Variable types: 0 continuous, 896 integer (236 binary)
Found heuristic solution: objectiv

In [513]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.ObjVal}")
else:
    print("No optimal solution found.")

Optimal objective value: 7353494.822999999


In [514]:
for i in range(0, len(df_DC_city)):
  print(z_var[f"z[{i}]"])

<gurobi.Var z[0] (value 1.0)>
<gurobi.Var z[1] (value 1.0)>
<gurobi.Var z[2] (value 0.0)>
<gurobi.Var z[3] (value 0.0)>


In [515]:
for i in range(0, len(df_DC_city)*len(df_customer_city)):
  print(y_var[f"y[{i}]"])

<gurobi.Var y[0] (value 0.0)>
<gurobi.Var y[1] (value 0.0)>
<gurobi.Var y[2] (value 0.0)>
<gurobi.Var y[3] (value 0.0)>
<gurobi.Var y[4] (value 0.0)>
<gurobi.Var y[5] (value 3600.0)>
<gurobi.Var y[6] (value 3700.0)>
<gurobi.Var y[7] (value 3800.0)>
<gurobi.Var y[8] (value 3900.0)>
<gurobi.Var y[9] (value 4000.0)>
<gurobi.Var y[10] (value 4100.0)>
<gurobi.Var y[11] (value 4200.0)>
<gurobi.Var y[12] (value 3000.0)>
<gurobi.Var y[13] (value 3200.0)>
<gurobi.Var y[14] (value 3300.0)>
<gurobi.Var y[15] (value 3400.0)>
<gurobi.Var y[16] (value 3500.0)>
<gurobi.Var y[17] (value 0.0)>
<gurobi.Var y[18] (value 0.0)>
<gurobi.Var y[19] (value 0.0)>
<gurobi.Var y[20] (value 0.0)>
<gurobi.Var y[21] (value 0.0)>
<gurobi.Var y[22] (value 0.0)>
<gurobi.Var y[23] (value 0.0)>
<gurobi.Var y[24] (value 0.0)>
<gurobi.Var y[25] (value 0.0)>
<gurobi.Var y[26] (value 0.0)>
<gurobi.Var y[27] (value 0.0)>
<gurobi.Var y[28] (value 0.0)>
<gurobi.Var y[29] (value 0.0)>
<gurobi.Var y[30] (value 0.0)>
<gurobi.Var y

In [516]:
for i in range(0, len(df_port_city)*len(df_DC_city)):
  print(x_var[f"x[{i}]"])


<gurobi.Var x[0] (value 27300.0)>
<gurobi.Var x[1] (value 16400.0)>
<gurobi.Var x[2] (value 0.0)>
<gurobi.Var x[3] (value 0.0)>


In [517]:
print(q)

0.0 + [ 211.00153 y[0] * z[0] + 254.29761000000005 y[1] * z[0] + 265.02621 y[2] * z[0] + 217.34354 y[3] * z[0] + 224.52512000000002 y[4] * z[0] + 8.316160000000002 y[5] * z[0] + 15.76321 y[6] * z[0] + 20.097260000000002 y[7] * z[0] + 25.857820000000004 y[8] * z[0] + 12.52304 y[9] * z[0] + 27.26346 y[10] * z[0] + 45.000730000000004 y[11] * z[0] + 33.89688 y[12] * z[1] + 22.80159 y[13] * z[1] + 19.78971 y[14] * z[1] + 51.37791 y[15] * z[1] + 20.842560000000002 y[16] * z[1] + 250.02857999999998 y[17] * z[1] + 235.59327999999996 y[18] * z[1] + 226.41069 y[19] * z[1] + 277.29097 y[20] * z[1] + 250.88106000000005 y[21] * z[1] + 225.96561000000003 y[22] * z[1] + 214.38341000000003 y[23] * z[1] + 188.71274 y[24] * z[2] + 210.9763 y[25] * z[2] + 229.79740000000004 y[26] * z[2] + 166.39556000000002 y[27] * z[2] + 209.06889999999999 y[28] * z[2] + 146.16813 y[29] * z[2] + 132.47031 y[30] * z[2] + 130.26391999999998 y[31] * z[2] + 172.75482 y[32] * z[2] + 159.97849 y[33] * z[2] + 153.513 y[34] * z

In [518]:
print(r)

43700


In [519]:
print(s)

10000.0 z[0] + 20000.0 z[1] + 2000000.0 z[2] + 2000000.0 z[3]


In [520]:
print(objective)

10000.0 z[0] + 20000.0 z[1] + 2000000.0 z[2] + 2000000.0 z[3] + [ 148.27279 x[0] * z[0] + 132.4207 x[1] * z[1] + 193.28627 x[2] * z[2] + 4.784680000000001 x[3] * z[3] + 211.00153 y[0] * z[0] + 254.29761000000005 y[1] * z[0] + 265.02621 y[2] * z[0] + 217.34354 y[3] * z[0] + 224.52512000000002 y[4] * z[0] + 8.316160000000002 y[5] * z[0] + 15.76321 y[6] * z[0] + 20.097260000000002 y[7] * z[0] + 25.857820000000004 y[8] * z[0] + 12.52304 y[9] * z[0] + 27.26346 y[10] * z[0] + 45.000730000000004 y[11] * z[0] + 33.89688 y[12] * z[1] + 22.80159 y[13] * z[1] + 19.78971 y[14] * z[1] + 51.37791 y[15] * z[1] + 20.842560000000002 y[16] * z[1] + 250.02857999999998 y[17] * z[1] + 235.59327999999996 y[18] * z[1] + 226.41069 y[19] * z[1] + 277.29097 y[20] * z[1] + 250.88106000000005 y[21] * z[1] + 225.96561000000003 y[22] * z[1] + 214.38341000000003 y[23] * z[1] + 188.71274 y[24] * z[2] + 210.9763 y[25] * z[2] + 229.79740000000004 y[26] * z[2] + 166.39556000000002 y[27] * z[2] + 209.06889999999999 y[28]

Visualization

In [521]:
!pip install folium
import folium

In [522]:
lat = 0
long = 0
n = 0
for i in range(0, len(df_coordinate_DC_city)):
  lat = lat + df_coordinate_DC_city['Latitude'][i]*z_var[f"z[{i}]"].x
  long = long + df_coordinate_DC_city['Longitude'][i]*z_var[f"z[{i}]"].x
  n = n + z_var[f"z[{i}]"].x
lat = lat/n
long = long/n
m = folium.Map(location=[lat, long], zoom_start=5.1)

In [523]:
for i in range(0, len(df_port_city)):
    # Providing location as a list [latitude, longitude]
    folium.Marker(
        location=[df_coordinate_port_city['Latitude'][i], df_coordinate_port_city['Longitude'][i]],
        popup=f"{df_coordinate_port_city['City'][i]}",
        icon=folium.Icon(color="darkgreen", icon="ship", prefix="fa")
    ).add_to(m)

In [524]:
for i in range(0, len(df_customer_city)):
    # Providing location as a list [latitude, longitude]
    folium.Marker(
        location=[df_coordinate_customer_city['Latitude'][i], df_coordinate_customer_city['Longitude'][i]],
        popup=f"{df_coordinate_customer_city['City'][i]}",
        icon=folium.Icon(color="red", icon="user", prefix="fa")
    ).add_to(m)

In [525]:
for i in range(0, len(df_DC_city)):
    # Providing location as a list [latitude, longitude]
    if z_var[f"z[{i}]"].x == 1:
      folium.Marker(location=[df_coordinate_DC_city['Latitude'][i], df_coordinate_DC_city['Longitude'][i]], popup=f"{df_coordinate_DC_city['City'][i]}",
        icon=folium.Icon(color="black", icon="warehouse", prefix="fa")).add_to(m)

In [526]:
for i in range(0, len(df_customer_city)):
    print(y_var[f"y[{i}]"].x)


0.0
0.0
0.0
0.0
0.0
3600.0
3700.0
3800.0
3900.0
4000.0
4100.0
4200.0


In [527]:
for i in range(0, len(df_port_city)*len(df_DC_city)):
  if z_var[f"z[{i}]"].x == 1:
    sourc_city = df_distance_port_DC['Source'][i]
    destination_city = df_distance_port_DC['Destination'][i]
    source_lat = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    source_lon = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]
    destination_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    destination_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]

    folium.PolyLine(
        locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
        color="darkgreen",
    ).add_to(m)

In [528]:
for i in range(0, len(df_DC_city)*len(df_customer_city)):
  if y_var[f"y[{i}]"].x > 1:
    # Get the source and destination cities
    source_city = df_distance_DC_customer['Source'][i]
    destination_city = df_distance_DC_customer['Destination'][i]

    # Get the coordinates for the source and destination cities
    source_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    source_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]
    destination_lat = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    destination_lon = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]

    # Add the polyline to the map
    folium.PolyLine(
        locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
        color="orange",
    ).add_to(m)

In [529]:
m